In [1]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3

UsageError: %%capture is a cell magic, but the cell body is empty.


In [ ]:
import boto3
import json

In [ ]:
# import keys from  local 

In [ ]:
s3resource = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'] )

In [ ]:
s3client = boto3.client(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'] )

In [ ]:
BUCKET = 'commonvoicesdataset'
FILE_TO_READ = 'vocab.json'
object1 = s3resource.Object(BUCKET,FILE_TO_READ)
file_content = object1.get()['Body'].read().decode('utf-8')
A =json.loads(file_content)

In [ ]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(A, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer('./vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from datasets.filesystems import S3FileSystem
s3fsystems = S3FileSystem(key=data['aws_access_key_id'], secret=data['aws_secret_access_key'])  

In [ ]:
from datasets import load_from_disk